In [1]:
%load_ext watermark
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from myst_nb import glue
from slugify import slugify

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.utils import resample

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

# from sklearn.model_selection import KFold

# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.svm import SVR

from plastockconf import name_zones, name_frequentation, name_situation
from plastockconf import name_substrate, name_distance, table_css_styles, table_css_styles_top

from plastock import add_table_to_page, capitalize_x_tick_labels, capitalize_x_and_y_axis_labels, capitalize_legend_components, attribute_summary

import reportclass as rc
import setvariables as conf_


import matplotlib as mpl

def translate_describe(x, value_column):
    described = x.to_dict()
    described.pop("count")
    described["moyenne"] = described.pop("mean")
    described["écart-type"] = described.pop("std")
    df = pd.DataFrame(described.items())
    df.set_index(0, inplace=True)
    df.rename(columns={1:value_column}, inplace=True)
    df.index.name = None
    
    
    return df


format_kwargs = dict(precision=2, thousands="'", decimal=",")
def make_exportable(data, file_name, cmap='YlOrBr'):
    data.fillna(0, inplace=True)
    fig, ax = plt.subplots(figsize=(12,8))
    sns.heatmap(data=data, vmin=0, vmax=1, cmap=cmap, annot=True, fmt='.2', annot_kws={'size':10}, ax=ax, cbar=False)
    plt.tight_layout()
    ax.tick_params(which='both', axis='both', bottom=False, left=False)
    plt.savefig(file_name, dpi=300)

    plt.close()

glue('blank_caption', " ", display=False)

In [2]:
new_data = pd.read_csv("data/end_pipe/macro_current.csv")
beach_data = pd.read_csv("data/pstock_beaches_current.csv")
codes = pd.read_csv('data/end_pipe/codes.csv').set_index('code')
language_maps = rc.language_maps()

new_column_names = {
    "Position":"position",
    "Substrat":"substrat",
    "Date":"date",
    "Code":"code",
    "Quantité":"quantité",
    "Aire":"area"
}

# import data and assign new column names and sample_id
# the sample_id is the tuple (location, date). Each row
# is a unique combinantion of sample_id and code
work_data = new_data[["Plage", *new_column_names.keys()]].copy()
work_data.rename(columns=new_column_names, inplace=True)
work_data["slug"] = work_data.Plage.apply(lambda x: slugify(x))
work_data["échantillon"] = list(zip(work_data.slug, work_data['date']))
work_data['date'] = pd.to_datetime(work_data["date"], format="mixed", dayfirst=True)
work_data.dropna(inplace=True)

# type the columns
work_data[["position", "substrat"]] = work_data[["position", "substrat"]].astype("int")
work_data['échantillon'] = work_data['échantillon'].astype(str)

# ! combine the surface areas of the position vectors !
# locate all the duplicate values by sample id and area
# this gives a data frame that has the position and area for
# each sample_id
total_area_dup = work_data.drop_duplicates(['échantillon', 'area'])

In [3]:
# sum of the areas for each position at each sample
# use the sample_id as index and sum the areas for each postition at each sample
total_area = total_area_dup.groupby('échantillon').area.sum()

# apply the total area to the work_data, index on sample_id
work_data['area_c'] = work_data['échantillon'].apply(lambda x: total_area.loc[x])
work_data['area'] = work_data.area_c
work_data.drop('area_c', axis=1, inplace=True)
# the code total per sample with the combined area
work_data = work_data.groupby(['échantillon', 'Plage', 'substrat', 'date', 'area','slug', 'quantité', 'code'], as_index=False)['quantité'].sum()
work_data.reset_index(inplace=True, drop=True)

# get the pcs/m²  for each object at each sample
work_data['pcs/m²'] = work_data['quantité']/work_data.area

In [4]:



# the code total per sample with the combined area
work_data = work_data.groupby(['échantillon', 'Plage', 'substrat', 'date', 'area','slug', 'quantité', 'code'], as_index=False)['quantité'].sum()
work_data.reset_index(inplace=True, drop=True)

# get the pcs/m²  for each object at each sample
work_data['pcs/m²'] = work_data['quantité']/work_data.area

# grouping on substrate should produce the same values as table A4-1 
test = work_data.groupby(['échantillon', 'substrat'], as_index=False)['pcs/m²'].sum()
dtest = test.groupby('substrat')['pcs/m²'].describe()
dtest.index.name = None
dtest.rename(columns={'count':'compte', 'mean':'moyenne', 'std':'écart type'}, inplace=True)
dtest['compte'] = dtest.compte.astype(int)

caption = "Les valeurs doivent correspondre au table A4-1 dans l'annexe 'Macro déchets plage et attribut'"

dtest = dtest.style.set_table_styles(table_css_styles).format(precision=2).set_caption(caption).format(**format_kwargs)

glue('dtest-sa', dtest, display=False)

In [5]:
# ! there are samples that have different substrates on the same sample_id.!
# there should be one substrate per sample_id. Identify the locations that have duplicate values
voi = 'substrat'
vals = "pcs/m²"
some_data = work_data.copy()
groupby = ['échantillon', voi]
data = some_data.groupby(groupby, as_index=False)[vals].sum()

# the samples with more than one substrate
dd = data[data['échantillon'].duplicated()].copy()

# select all the duplicated sample_ids from the work_data
duplicated = work_data[work_data['échantillon'].isin(dd['échantillon'].unique())].copy()
# change the substrat to [2]
duplicated['substrat'] = 2 

# select all the values that are not duplicated
not_duplicated = work_data[~(work_data['échantillon'].isin(dd['échantillon'].unique()))].copy()

# put it back together again
work_data = pd.concat([duplicated, not_duplicated])

# ! valid codes and definitions !
# plastock did not use the same inventory as iqaasl
# here we select only the codes in the plastock inventory
pcodes = work_data.code.unique()

# identify and remove codes for which there is no defintion
# if the code is not defined then it can not be used
t = [x for x in pcodes if x not in codes.index]
wd_ni = work_data[~work_data.code.isin(t)].copy()

# ! aggregating to Gfrags, Gcaps and Gfoams !
# these items are not well divided into the composite subgroups
# for example people often know what a cap is, but whether it 
# comes from a drink bottle or other type is not well considered
# we combine the subcategories into more comprehensive groups.
ti = rc.use_gfrags_gfoams_gcaps(wd_ni, codes)

# ! groupby the sample id otherwise there are duplicate codes
# after aggregating to Gfrags etc..
work_data = ti.groupby(['échantillon', 'Plage', 'substrat', 'date', 'area', 'slug', 'code'], as_index=False).agg({'quantité':'sum'})
work_data['pcs/m²'] = work_data['quantité']/work_data['area']

# accounting for objects not found at a sample:
# the codes that were indentified are the unique codes
# in the set of data
codes_ip = work_data.code.unique()
# the unique samples by id
loc_dates = work_data['échantillon'].unique()

# a copy for itterating
wd = work_data.copy()

# for each sample (échantillon) indentify the codes that were not
# found by indentifying all the codes that were found and removing 
# them from the list of all unique codes.
# for each unidentified code per sample, add a row with the sample
# id and the code. give the row a quantity of zero.
rows = []
for a_loc in loc_dates:
    r = wd.loc[wd['échantillon'] == a_loc].copy()
    r.reset_index(inplace=True, drop=True)
    
    t = r.loc[0][['échantillon', 'Plage', 'substrat', 'date', 'area', 'slug']].values
    asamp = [x for x in t]
    used_codes = r.code.unique()
    unused = [x for x in codes_ip if x not in used_codes]
    for element in unused:
        arow = [*asamp, element, 0, 0]
        rows.append(arow)
        

# now all the data has the same number of records per sample
# for each sample we can now say what was found and what was
# not found with respect to all the results
work_x = pd.DataFrame(rows, columns=['échantillon', 'Plage', 'substrat', 'date', 'area', 'slug', 'code', 'quantité', 'pcs/m²'])
work_data = pd.concat([work_x, work_data])


# add the regional component
regions = pd.read_csv("data/end_pipe/lac_leman_regions.csv")
regions.loc[regions.slug == 'savoniere', 'slug'] = 'savonniere'
regions.drop_duplicates('slug', inplace=True)
regions.set_index('slug', drop=True, inplace=True)
work_data['region'] = work_data.slug.apply(lambda x: regions.loc[x, 'alabel'])
# the city names are not included with the plastock data
# they have been incorporated into the IQAASL data
# we need to change the names from plastock that are duplicates
# in the IQAASL data.
change_names = ['preverenges', 'tolochenaz', 'versoix', 'vidy', 'cully']

changeus = work_data[work_data.slug.isin(change_names)].copy()
donotchange = work_data[~work_data.slug.isin(change_names)].copy()

new_slug = {
    'cully': 'cully-p',
    'preverenges': 'preverenges-p',
    'tolochenaz': 'tolochenaz-p',
    'versoix':'versoix-p',
    'vidy': 'vidy-p'}

# they have the same name as locations in iqaasl
changeus['new_slug'] = changeus.slug.apply(lambda x: new_slug[x])
changeus['slug'] = changeus.new_slug
changeus.drop('new_slug', inplace=True, axis=1)

# the plastock data with the converted names
report_data = pd.concat([changeus, donotchange])

f = pd.read_csv('data/u_pstk.csv')
city_map = f[["slug", "city"]].drop_duplicates()
city_map.loc[city_map.slug == 'savoniere', 'slug'] = 'savonniere'
city_map.set_index('slug', inplace=True)

# adding and renaming columns according to reportclass requirements
# these values can be indexed on the IQAASL data
report_data['city'] = report_data.slug.apply(lambda x: city_map.loc[x])
report_data['groupname'] = report_data.code.apply(lambda x: codes.groupname.loc[x])

In [6]:
def name_the_new_distance(x, less='<= 500 m', more = '> 500 m'):
    if x == 1:
        return less
    else:
        return more

def name_the_new_freq(x, new):
    if x <= 2:
        return new
    else:
        return 'Elévée'


# the feature variables are added to the work_data
ti = work_data.copy()
features = ['frequentation', 'situation', 'orientation', 'distance',]

beach_datax = pd.read_csv("data/end_pipe/asl_beaches.csv").set_index('Plage')

# they can be merged on the Plage column and the index
env_plastock = ti.merge(beach_datax[features], left_on='Plage', right_index=True)

# ! creation of composite variables !
t_and_f = env_plastock.loc[:, ['échantillon', 'slug','date','code', 'pcs/m²', 'quantité', 'frequentation', 'situation', 'distance', 'substrat', 'region']].copy()

# the substrat and distance features are being combined
# the two lowest and the two highest of each group are being combined
# substrat is a matter of combining different granularities. They are being grouped as
# sand and gravel.
# distance is now grouped by locations either less than or equal to 500 meters
t_and_f.loc[t_and_f.substrat <= 2, 'substrat'] = 1
t_and_f.loc[t_and_f.substrat > 2, 'substrat'] = 2
t_and_f.loc[t_and_f.distance <= 2, 'distance'] = 1
t_and_f.loc[t_and_f.distance > 2, 'distance'] = 2
t_and_f.loc[t_and_f.frequentation <= 2, 'frequentation'] = 2

# ! the data used in the models !
f_combi = t_and_f.copy()

f_combi.rename(columns={'frequentation':'fréquentation', 'loc_date': 'échantillon'}, inplace=True)

# the feature variables are combined along the ordinal axis. Going from four catgories
# to two in the case of distance and substrate. city and country are already binary
# the values of low and moderate frequentation are combined also.
f_comb = f_combi.copy()
f_comb['distance'] = f_comb['distance'].apply(lambda x: name_the_new_distance(x))
f_comb['fréquentation'] = f_comb['fréquentation'].apply(lambda x: name_the_new_freq(x, 'faible-moyenne'))
f_comb['situation'] = f_comb['situation'].apply(lambda x: name_situation[x])
f_comb['substrat'] = f_comb['substrat'].apply(lambda x: name_the_new_distance(x, less='Sable', more='Graviers'))

# ! no composite variables !
no_combined = env_plastock.loc[:, ['échantillon', 'slug','date','code', 'pcs/m²', 'frequentation', 'situation', 'distance', 'substrat', 'region']].copy()
no_combined.rename(columns={'frequentation':'fréquentation', 'loc_date': 'échantillon'}, inplace=True)

no_combined['distance'] = no_combined['distance'].apply(lambda x: name_distance[x])
no_combined['fréquentation'] = no_combined['fréquentation'].apply(lambda x: name_frequentation[x])
no_combined['situation'] = no_combined['situation'].apply(lambda x: name_situation[x])
no_combined['substrat'] = no_combined['substrat'].apply(lambda x: name_substrate[x])

In [7]:
fig, axs = plt.subplots(2, 2, figsize=(8,8))
wk_dt = report_data.groupby(['échantillon', 'date'], as_index=False)['pcs/m²'].sum()
vals = 'pcs/m²'
object_column = vals
ylim = 4
xlim = 4
sns.scatterplot(wk_dt, x="date", y=vals, label='Plastock 2022 m²', ax=axs[0, 0])
sns.boxplot(wk_dt, y=vals,  showfliers=True, ax=axs[0, 1], dodge=False)
sns.histplot(wk_dt, x=vals,  ax=axs[1, 0], stat='probability', kde=True)
sns.ecdfplot(wk_dt, x=vals,  ax=axs[1, 1])

axs[0,0].xaxis.set_major_locator(mdates.MonthLocator(interval=3))
axs[0,0].xaxis.set_major_formatter(mdates.DateFormatter('%m-%y'))

axs[0, 0].set_ylim(-.01, ylim)
axs[0, 1].set_ylim(-.01, ylim)
axs[1, 1].set_xlim(-.01, xlim)
axs[1, 0].set_xlim(-.01, xlim)

axs[0, 0].set_xlabel("date")
axs[0, 0].set_ylabel(object_column)

axs[1, 0].set_xlabel(object_column)
axs[1, 0].set_ylabel("Probabilité")
axs[0, 1].set_xlabel("")
axs[0, 1].set_ylabel(object_column)
axs[0, 1].tick_params(axis="x", rotation=45)

axs[1, 1].set_xlabel(object_column)
axs[0,0].set_title("Total par échantillon", loc="left")
axs[0,1].set_title("Boîte de Tukey", loc="left")
axs[1,0].set_title("Histogramme", loc="left")
axs[1,1].set_title("Fonction de répartition", loc="left")
plt.subplots_adjust(wspace=.3)


plt.tight_layout()

glue('situataion-sa', fig, display=False)
plt.close()

In [8]:
situation_summary = rc.a_summary_of_one_vector(report_data, ['échantillon', 'date'], {'pcs/m²':'sum', 'quantité':'sum'}, describe  ='pcs/m²', total_column='quantité')
situation_summary.rename(columns={'pcs/m²':'résultats'}, inplace=True)
sit_disp = rc.translated_and_style_for_display(situation_summary, language_maps['fr'], 'fr', gradient=False)
glue('sit_display', sit_disp, display=False)

In [9]:
# most common

quantity_code = work_data.groupby('code')['quantité'].sum()
pcsm_code = work_data.groupby('code')['pcs/m²'].mean()
percent_total = quantity_code/quantity_code.sum()

# fail_rate
work_data['gzero'] = work_data['quantité'] > 0
fail_rate = work_data.groupby('code').gzero.sum()
fail_rate = fail_rate/work_data['échantillon'].nunique()
inventory = pd.DataFrame({'Quantité':quantity_code, '% du total':percent_total, 'pcs/m²':pcsm_code, 'Taux d\'échec':fail_rate})
abundant_codes = inventory.sort_values(by='Quantité', ascending=False)[:10].index
frequent_codes = inventory[inventory['Taux d\'échec'] >= 0.5].index
most_common_codes = list(set([*frequent_codes, *abundant_codes]))

most_common_codes_df = inventory.loc[most_common_codes].sort_values(by='Quantité', ascending=False)
most_common_codes_df['index'] = most_common_codes_df.index.map(lambda x: codes.fr.loc[x])

mcm = most_common_codes_df.set_index('index', drop=True)
mcm.index.name = None
mc = mcm.style.set_table_styles(table_css_styles).format(**format_kwargs)

glue('mcm-sa', mc, display=False)

In [10]:
w_df = report_data.copy()
cities = w_df.city.unique()
cities = w_df.city.unique()
cities.sort()
cone = cities[:12]
ctwo = cities[12:]



sample_id = "échantillon"
group_agg = {"quantité":"sum", "pcs/m²":"mean"}
unit_agg = {"quantité":"sum", "pcs/m²":"sum"}
pivot_values = vals

args = dict(
    feature_name='city', 
    object_column='code',
    sample_id=sample_id, 
    group_agg=group_agg, 
    unit_agg=unit_agg, 
    pivot_values=vals, 
    table_split=cone
)

t = rc.a_cumulative_report(w_df[(w_df.code.isin(most_common_codes))], **args)

caption_tone = "Les résultats des objets les plus courants en pcs/m² pour chaque ville du projet: Amphion à Hermance"

mc_c1 = rc.translated_and_style_for_display(t, language_maps['fr'], 'fr',  gradient=True).set_caption(caption_tone)
glue('most_common_c1', mc_c1, display=False)

In [11]:
file_name = 'resources/images/most_common_one-sa.jpg'
make_exportable(t, file_name)

In [12]:
args.update({'table_split':ctwo})
t = rc.a_cumulative_report(w_df[(w_df.code.isin(most_common_codes))], **args)

caption_ttwo = "Les résultats des objets les plus courants en pcs/m² pour chaque ville du projet: Lugrin à Vidy"

mc_c2 = rc.translated_and_style_for_display(t, language_maps['fr'], 'fr',  gradient=True).set_caption(caption_ttwo)
glue('most_common_c2', mc_c2, display=False)

In [13]:
file_name = 'resources/images/most_common_two-sa.jpg'
make_exportable(t, file_name)

In [14]:
w_df = report_data.copy()
cities = w_df.city.unique()
cities.sort()
cone = cities[:12]
ctwo = cities[12:]

args = dict(
    feature_name='city', 
    object_column='groupname', 
    sample_id=sample_id, 
    group_agg=group_agg, 
    unit_agg=unit_agg, 
    pivot_values=vals, 
    table_split=cone
)

caption_gn1 = "Les résultats des objets par utilisation pour chaque ville du projet: Amphion à Hemance"

groups_df = report_data.copy()
group_name_map = codes['groupname']
groups_df['groupname'] = groups_df.code.apply(lambda x: group_name_map.loc[x])
tg1 = rc.a_cumulative_report(groups_df, feature_name='city', object_column='groupname', sample_id=sample_id, group_agg=group_agg, unit_agg=unit_agg, pivot_values=vals, table_split=cone)

gn_c1 = rc.translated_and_style_for_display(tg1, language_maps['fr'], 'fr', gradient=True).set_caption(caption_gn1)
glue('gn_c1', gn_c1, display=False)

In [15]:
file_name = 'resources/images/group_names_one-sa.jpg'
make_exportable(tg1, file_name)

In [16]:
args.update({'table_split':ctwo})

caption_gn2 = "Les résultats des objets par utilisation pour chaque ville du projet: Lugrin à Vevey"
tg2 = rc.a_cumulative_report(groups_df, **args)
gn_c2 = rc.translated_and_style_for_display(tg2, language_maps['fr'], 'fr', gradient=True).set_caption(caption_gn2)

glue('gn_c2', gn_c2, display=False)

In [17]:
file_name = 'resources/images/group_names_two-sa.jpg'
make_exportable(tg2, file_name)

In [18]:
unit_columns = ['region', 'échantillon']
rg = []
label='Haut lac'
summary_index = ['min', '25%', '50%', '75%', 'max', 'mean', 'std', 'count', 'total']
for label in report_data.region.unique():
    g = rc.a_summary_of_one_vector(report_data[report_data.region == label].copy(), unit_columns=unit_columns, unit_agg=unit_agg, describe='pcs/m²', label=label, total_column='quantité')
    rg.append(g)
regional_summary = pd.concat(rg)
rg = regional_summary.pivot(columns='label', values='pcs/m²')
rg = rg.reindex(summary_index)

rcg = rc.translated_and_style_for_display(rg, language_maps['fr'], 'fr', gradient=False)

caption = "Distribution des résultats par échantillon et par région : Plastock 2022"

rcg_hist = rcg.highlight_max(axis=1, props= 'color: rgba(255, 0, 0, 1);').set_caption(caption)
glue('regional_summary_sa', rcg_hist, display=False)

In [19]:
mc_reg = report_data.groupby(['region', 'échantillon', 'code'], as_index=False)['pcs/m²'].sum()
mcg = mc_reg[mc_reg.code.isin(most_common_codes)].groupby(["region", "code"], as_index=False)["pcs/m²"].mean()
mcg = mcg[['region', 'code', 'pcs/m²']].pivot(index='code', values='pcs/m²', columns='region')

# language_maps = rc.language_maps()
mcg = rc.translated_and_style_for_display(mcg, language_maps['fr'], 'fr', gradient=False) 

caption = "La moyenne des objets les plus courants par région. Plastock 2022"
mcg_hist = mcg.highlight_max(axis=1, props= 'color: rgba(255, 0, 0, 1);').set_caption(caption)
glue('regional_most_common', mcg_hist, display=False)

In [20]:
distance_index = ['<= 500 m', '> 500 m']
freq_index = ['Elévée', 'faible-moyenne']
sub_index =  ['Graviers', 'Sable']
def reindex_df(category, df, index):
    df = df.reindex(index=index)
    return df

def calculate_combined_stats(category, data=f_comb.copy(), index=None):
    # Descriptive statistics of the sample density and quantity of pieces found
    # Aggregates the data on category and sample_id    
    grouped = data.groupby([category, 'échantillon'], as_index=False)['pcs/m²'].sum()

    group_summary = grouped.groupby(category, as_index=True).agg({'échantillon':'nunique', 'pcs/m²':['mean', 'median']})
    group_summary.columns = group_summary.columns.droplevel(0)
    
    # Calculating percentage of total samples
    group_summary['percentage'] = (group_summary['nunique'] / data['échantillon'].nunique()) * 100
    group_summary.reset_index(drop=False, inplace=True)

    # Renaming columns for clarity
    group_summary.rename(columns={'nunique': 'échantillons','mean': 'Moyenne', 'median': 'Médiane', 'percentage': '%'}, inplace=True)
    group_summary.set_index(category, inplace=True, drop=True)
    
    # Make the index labels if required
    if index is not None:
        group_summary = reindex_df(category, group_summary, index=index)
    
    
    return group_summary.style.set_table_styles(table_css_styles).format(precision=2).hide(axis=0, names=True)

# Calculate stats for each category with combined rows
frequentation_stats = calculate_combined_stats('fréquentation', index=freq_index)
situation_stats = calculate_combined_stats('situation')
distance_stats = calculate_combined_stats('distance', index=distance_index)
substrat_stats = calculate_combined_stats('substrat', index=sub_index)
section = 'MD'
page = '1'

freq_stats = add_table_to_page(frequentation_stats, 2, ": Densité de déchets par fréquentation", section, page, "", format_index=None)
sit_stats = add_table_to_page(situation_stats, 3, ": Densité de déchets par urbanization", section, page, "", format_index=None)
dist_stats = add_table_to_page(distance_stats, 4, ": Densité de déchets par distance de parking", section, page, " ", format_index=None)
sub_stats = add_table_to_page(substrat_stats, 5, ": Densité de déchets selon substrat", section, page, "", format_index=None)

glue('freq_stats_sa', freq_stats, display=False)
glue('sit_stats_sa', sit_stats, display=False)
glue('dist_stats_sa', dist_stats, display=False)
glue('sub_stats_sa', sub_stats, display=False)

# Plastock Macros déchets 

Les déchets de plage ont été échantillonnés tous les trimestres du 1er janvier 2022 au 31 décembre 2022 sur les plages du lac Léman. Les participants ont mesuré la longueur le long de la ligne d'eau et la largeur, la distance de la ligne d'eau, de chaque site d'étude. Au cours de cette période de 12 mois, les volontaires ont collecté et identifié 27 493 objets sur 25 sites, représentant 24 villes du bassin du Léman, ([résultats](sa_results)).

Le résultat moyen était de 0,66 déchets par mètre² (pcs/m²). L'objet le plus courant est le plastique fragmenté, qui représente 41 % du total des objets (0,26 pièce/m²), suivi des mégots de cigarettes, qui représentent 11 % du total (0,07 pièce/m²).  Les objets les plus courants, ([plus courants](mc_sa)), en 2022 avaient tous été identifiés lors de campagnes antérieures sur le lac. Il convient de noter que six des objets les plus courants identifiés dans les études Plastock sont également les plus courants dans la région OSPAR ([OSPAR correspondance](cor_ospar)). 

Les échantillons ont été prélevés des deux côtés du lac et dans les trois régions. Le plus grand nombre d'échantillons a été collecté dans le Grand lac (47), suivi du Petit lac (27) et enfin du Haut lac (24). Le résultat moyen était le plus élevé dans le Haut lac (0,95 pcs/m²), suivi par le Grand lac (0,65 pcs/m²) et enfin le Petit lac (0,4 pcs/m²). Tous les objets les plus courants, à l'exception des capsules de bouteilles, ont également enregistré des valeurs moyennes plus élevées dans le Haut Lac. Cependant, la valeur médiane est la plus élevée dans le Grand lac et la différence entre la médiane et la moyenne est la plus élevée dans le Haut lac ([régionalité](mc_regional)).

Les résultats de l'enquête sur les objets tels que les mégots de cigarettes et les emballages de snacks devraient être plus élevés dans les endroits où le substrat est sablonneux et où il y a un accès à un parking. Les sites éloignés ou constitués d'un substrat de gravier présentent des niveaux plus élevés de plastiques fragmentés. Différents modèles ont été envisagés à l'aide d'une régression par forêt aléatoire et d'une approximation par grille([random forest](random_forest_sa)) and a grid approximation ([grid approximation](grid_approx_p)).

__Autres campganes :__

Le Swiss Litter Report ([Swiss Litter Report](http://www.stoppp.org/)) fait état d'une moyenne de 0,67 déchet par m². La moyenne déclarée pour les lacs était de 1,23 pcs/m², avec des chiffres élevés dans les zones urbaines de 1,03 pcs/m². Nous notons que ces valeurs se situent dans la fourchette que l'on pourrait attendre de la mer ionienne (0,67 pcs/m²) ou de la mer méditerranée (0,61 pcs/m²).

__Perspectives 2024 :__

Si nous utilisons la même logique que celle appliquée dans ([résulats précédents](previous_results)), le résultat médian estimé pour 2024 se situe entre 0,43 pcs/m² et 0,62 pcs/m². Les sites avec un substrat de gravier ou plus éloignés auront des valeurs de comptage plus faibles pour les mégots de cigarettes et les emballages de snacks, mais des proportions plus élevées de plastiques fragmentés.

(sa_results)=
## Résultats
::::{grid} 1 1 2 2

:::{grid-item}

{glue}`sit_display`

:::

:::{grid-item}
L’échantillonnage était planifié trimestriellement, commençant en janvier 2022. À quelques exceptions près, chaque emplacement était échantillonné quatre fois au cours de la période de 12 mois.

Au total, 98 échantillons ont été enregistrés sur 25 sites représentant 24 villes sur les rives du Léman.
:::

:::{grid-item}
:columns: 12
{glue}`situataion-sa`
:::
::::


(mc_sa)=
### Les objets les plus courants 

#### Définition des _objets les plus courants_

Les _objets les plus courants_ peuvent être sélectionnés de plusieurs manières. On peut également les appeler les _objets d'intérêt_. Dans le cadre de ce rapport, nous nous concentrons sur les objets qui représentent une proportion plus importante des résultats que les autres. Nous avons utilisé deux critères de sélection : i. la quantité, ii. le taux d'échec.

1. Quanité: Si un objet a une quantité totale qui le place dans les dix premiers, il est considéré comme 'courant'.
2. Taux d'échec: Si un objet a été trouvé dans au moins la moitié des échantillons, il est ÉGALEMENT considéré comme 'courant'.

Par conséquent, pour cette étude, les 'objets les plus courants' sont ceux qui se trouvent soit dans les dix premiers en termes de nombre total de pièces de déchets ET/OU qui ont été trouvés dans au moins 50% des enquêtes. Pour Plastock, les objets les plus courants représentent 89% du montant total ou 24'156/27'493 [Les plus courants](most_common_p).

{glue}`mcm-sa`

(mc_regional)=
#### Les plus courants par région


::::{grid} 1 1 2 2
:::{grid-item}
{glue}`regional_summary_sa`
:::

:::{grid-item}
:padding: 4 1 1 1
Comme nous l'avons vu précédemment, le Haut Lac a en moyenne des valeurs plus élevées que les deux autres régions. Cependant, nous remarquons que les valeurs minimales et de 25 % sont plus faibles dans le Haut Lac et que la médiane est presque la même que dans le Grand Lac. Cela suggère que les événements extrêmes sont plus probables dans le Haut Lac que dans le Grand Lac.
:::

:::{grid-item}
:columns: 12

{glue}`regional_most_common`

:::
::::


#### Les plus courants par ville

__Amphion à Hermance__

```{glue} most_common_c1
```
<br>

__Lugrin à Vevey__

```{glue} most_common_c2
``` 

### Les objets trouvés en fonction de leur utilisation

Le type d'utilité est basé sur l'utilisation de l'objet avant qu'il ne soit jeté ou sur la description de l'objet si l'utilisation initiale est indéterminée. Les objets identifiés sont classés dans l'une des 260 catégories prédéfinies. Les catégories sont regroupées en fonction de leur utilisation ou de leur description.

- Eaux usées : objets rejetés par les stations d'épuration, y compris les objets susceptibles d'être jetés dans les toilettes.
- Microplastiques (< 5 mm) : plastiques fragmentés et résines plastiques de préproduction.
- Infrastructure : objets liés à la construction et à l'entretien des bâtiments, des routes et des réseaux d'eau et d'électricité.
- Alimentation et boisson : tous les matériaux liés à la consommation de nourriture et de boissons.
- Agriculture : principalement des feuilles industrielles, par exemple, paillis et bâches de culture, serres, fumigation du sol, films d'emballage de balles. Comprend les plastiques durs pour les clôtures agricoles, les pots de fleurs, etc.
- Tabac : principalement des filtres de cigarettes, y compris tous les matériaux liés au tabagisme.
- Loisirs : objets liés au sport et aux loisirs, par exemple, pêche, chasse, randonnée, etc.
- Emballages non alimentaires et non liés au tabac : matériaux d'emballage non identifiés comme étant liés à la nourriture, aux boissons ou au tabac.
- Fragments de plastique : morceaux de plastique d'origine ou d'utilisation indéterminée.
- Objets personnels : accessoires, articles d'hygiène et vêtements.

Pour des informations détaillées sur la composition des groupes, consultez [IQAASL - DE](https://hammerdirt-analyst.github.io/IQAASL-End-0f-Sampling-2021/code_groups.html) ou [IQAASL - EN](https://www.plagespropres.ch/code_groups.html).
<br>

__Amphion à Hermance__

```{glue} gn_c1
```
<br>

__Lugrin à Vevey__

```{glue} gn_c2
``` 
<br>

### Différences entre les types de plage

Les conditions d’échantillonnage

Les enquêteurs ont classé les conditions à chaque emplacement d’échantillonnage selon quatre catégories, fréquentation, substrat, distance au parking et situation (ville/campagne). Les plages ayant un taux d’utilisation élevé représentaient 56% (55/98) de tous les échantillons. Les plages situés à moins de 500 mètres d’un parking représentent 84% (83/98). Le résumé en détail est disponible ([attributes](macro-attributes)).

Les catégories ont été combinées afin de réduire la covariance et d'améliorer l'interprétabilité. Les catégories ont été réduites de 4 à 2 dans le cas du substrat et de la distance au parking. La fréquence a été réduite à deux groupes en combinant les deux rangs les plus bas. Vous pouvez consulter les détails de la raison et de la manière dont nous procédons ici.: ([correlations](correlations)). Les résultats sont présentés ci-dessous :

::::{grid} 1 1 2 2 

:::{grid-item}

{glue}`freq_stats_sa`

:::

:::{grid-item}

{glue}`dist_stats_sa`

:::

:::{grid-item}

{glue}`sub_stats_sa`

:::

:::{grid-item}

{glue}`sit_stats_sa` 

:::

::::

Les lieux avec une plage de sable avaient la valeur médiane la plus élevée, soit .52 pcs/m² v/s graviers 0.24 pcs/m². Les lieux classés comme urbains avaient une valeur médiane de 0,52 pcs/m² pour 31 échantillons, contre 0,38 pcs/m² a la campagne pour 67 échantillons. Mais, l’échantillon moyenne attendu est plus élevé dans les localités désignées comme étant à la campagne (0.71 contre 0.57). Á la campagne, on s’attend à trouver plus de Gfrags et de pellets industriels (GPI) en % du total. Dans les villes, nous nous attendons à trouver plus de bouts de cigarettes et d’emballages de snacks par échantillon. Des prédictions ont été faites en utilisant différentes combinaisons de catégories à l’aide du régresseur de la forêt aléatoire ([Random forest](random_forest_sa)) et d’une approximation de la grille bayésienne ([Grid approximations](grid_approx_p)).




<!-- ### Exigences particulières données plastock

La quantité de déchets sauvages par mètre² de plage correspond au nombre total d'objets identifiés divisé par la surface d'étude. Pour les données Plastock, cela signifie que nous devons considérer la position un (ligne d'eau) et la position deux (plage sèche) ensemble. 


::::{grid}

:::{grid-item}
{glue}`dtest-sa`
:::

:::{grid-item}
:padding: 4 2 2 2
Les déchets par mètre carré sont considérés comme la somme de la surface de la position 1 et de la surface de la position 2 pour chaque échantillon [Macro déchets plage et attribut](macro-attributes).
:::
::::

De plus, il y a __douze échantillons où le substrat était différent entre la position un et la position deux__. Par conséquent, ces 12 échantillons ont tous été classés dans la catégorie Sables grossiers. La distribution de la variable de substrat après la réattribution des 12 échantillons en question : 1. Sables fins : 27%, 2. Sables Grossiers : 32%, 3. Graviers : 16%, 4. Cailloux : 25%. -->



## Estimation des paramètres et prédictions

In [21]:
def analyze_scenario(scenario_data, func, n_iterations=100, bin_width=0.2):
    """
    Analyze a specific scenario using Random Forest regression with bootstrapping,
    and calculate feature importances.

    :param data: DataFrame containing the dataset.
    :param feature_1: The name of the first feature for filtering.
    :param feature_1_value: The value of the first feature to filter by.
    :param feature_2: The name of the second feature for filtering.
    :param feature_2_value: The value of the second feature to filter by.
    :param n_iterations: Number of bootstrap iterations. Default is 100.
    :param bin_width: Width of each bin for histogram. Default is 0.2.
    :return: A tuple containing bins, bin probabilities, flattened predictions, and feature importances.
    """
    
    # Prepare data for regression
    y_scaler = MinMaxScaler()
    y_scaled = y_scaler.fit_transform(scenario_data['pcs_m'].values.reshape(-1,1)).flatten()
    
    # Initialize the OneHotEncoder
    # here we encode the ordinal data
    encoder = OneHotEncoder(sparse_output=False)
    
    X = scenario_data.drop('pcs_m', axis=1)
    
    # Apply the encoder to the categorical columns
    encoded_data = encoder.fit_transform(scenario_data[['fréquentation', 'situation', 'distance', 'substrat']])
    # Create a DataFrame with the encoded data
    X_encoded = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['fréquentation', 'situation', 'distance', 'substrat']))

    
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_scaled, test_size=0.2, random_state=42)

    # Bootstrap predictions and accumulate feature importances
    bootstrap_predictions = []
    feature_importances_accumulated = np.zeros(X_train.shape[1])
    
    # Collect diagnostic at each repetition
    cum_mse = []
    cum_r2 = []
    
    for _ in range(n_iterations):
        X_train_sample, y_train_sample = resample(X_train, y_train)
        rf_model_sample = func
        rf_model_sample.fit(X_train_sample, y_train_sample)
        
        pred = rf_model_sample.predict(X_test)
        
        r2 = r2_score(y_test, pred)
        pred = y_scaler.inverse_transform(pred.reshape(-1, 1)).flatten()
        bootstrap_predictions.append(pred)
        mse = mean_squared_error(y_test , pred)
        
        
        feature_importances_accumulated += rf_model_sample.feature_importances_
        
        cum_mse.append(mse)
        cum_r2.append(r2)

        # Average feature importances
    feature_importances = feature_importances_accumulated / n_iterations

    # Flatten the predictions array
    predictions_flat = np.array(bootstrap_predictions).flatten()

    return predictions_flat, feature_importances, cum_mse, cum_r2

def plot_histogram(predictions, observed, title="", reference='camp-dist-1', display=False):
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.histplot(predictions, bins=20, stat="probability", ax=ax, label='prédictions', zorder=0)
    sns.histplot(observed, bins=20, stat="probability", label='observée', zorder=1, ax=ax)
    plt.title(title, loc='left')
    plt.xlabel('pcs/m')
    plt.ylabel('Densité de Probabilité')
    plt.legend()
    glue(reference, fig, display=display)
    plt.close()

def evalutate_model(r2s, mses, label, model='random-forest'):
    r2 = np.round(np.mean(r2s), 2)
    mse = np.round(np.mean(mses), 2)
    results = {"cross validated error":r2, "mean² error":mse, 'model':model}
    return pd.DataFrame(results, index=[label])

# Calculating quantiles for Scenario 2
q_uants = [0.01, 0.25, 0.5, 0.75, 0.99]
index = ['1%', '25%', '50%', '75%', '99%', 'Moyenne']
def makeqdf(observed, predicted, index=index, quants=q_uants, caption=""):
    
    o_q = np.quantile(observed, quants)
    m_o = np.mean(observed)
    o_p = np.quantile(predicted, quants)
    m_p = np.mean(predicted)
    
    results = {'observée':[*o_q, m_o], 'prédiction': [*o_p, m_p]}
    return pd.DataFrame(results, index=index).style.set_table_styles(table_css_styles_top).format(precision=2).set_caption(caption)

cols = ['échantillon', 'fréquentation','situation', 'distance', 'substrat']

In [22]:
# Filter for Scenario 
test_xi = f_combi[(f_combi['situation'] == 2) & (f_combi['fréquentation'] == 3)].copy()
test_xi.rename(columns={'pcs/m²':'pcs_m'}, inplace=True)
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()
test_x = test_x[['fréquentation', 'situation', 'distance', 'substrat', 'pcs_m']]

# model parameters
estimators = 10
iterations = 1000

func = RandomForestRegressor(n_estimators=estimators, criterion="absolute_error", random_state=42)
predictions, feature_importance, mse, r2 = analyze_scenario(test_x, func,  n_iterations=iterations, bin_width=0.2)

caption = 'Urban, Fréquentation Elévée'
q_sit_2_freq_3 = makeqdf(test_x.pcs_m.values, predictions, caption=caption)
glue('q-hf-ville-sa', q_sit_2_freq_3, display=False)

# the histogram for this scenario:
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions - Situation Ville, Haute Fréquentation'
plot_histogram(predictions, test_x.pcs_m.values, title=title, reference='ville-hf-sa', display=False)

In [23]:
# Filter for Scenario 
test_xi = f_combi[(f_combi['situation'] == 1) & (f_combi['fréquentation'] == 3)].copy()
test_xi.rename(columns={'pcs/m²':'pcs_m'}, inplace=True)
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()
test_x = test_x[['fréquentation', 'situation', 'distance', 'substrat', 'pcs_m']]

func = RandomForestRegressor(n_estimators=estimators, criterion="absolute_error", random_state=42)
predictions, feature_importance, mse, r2 = analyze_scenario(test_x, func,  n_iterations=iterations, bin_width=0.2)

# the quantiles for this scenario
caption="Campagne, Fréquentation Eléveé"
q_sit_1_freq_3 = makeqdf(test_x.pcs_m.values, predictions, caption=caption)
glue('q-hf-camp-sa', q_sit_1_freq_3, display=False)

# the histogram for this scenario:
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions - Situation Campagne, Haute Fréquentation\n'
plot_histogram(predictions, test_x.pcs_m.values, title=title, reference='camp-hf-sa', display=False)

In [24]:
# Filter for Scenario 
test_xi = f_combi[(f_combi['situation'] == 1) & (f_combi['distance'] == 1)].copy()
test_xi.rename(columns={'pcs/m²':'pcs_m'}, inplace=True)
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()
test_x = test_x[['fréquentation', 'situation', 'distance', 'substrat', 'pcs_m']]


func = RandomForestRegressor(n_estimators=estimators, criterion="absolute_error", random_state=42)
predictions, feature_importance, mse, r2 = analyze_scenario(test_x, func,  n_iterations=iterations, bin_width=0.2)

# the quantiles for this scenario
caption = 'Campagne, <= 500 m du parking'
q_sit_1_d_1 = makeqdf(test_x.pcs_m.values, predictions, caption=caption)
glue('q-camp-dist_1-sa', q_sit_1_d_1, display=False)

# the histogram for this scenario:
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions - Situation Campagne, distance < 500 m'
plot_histogram(predictions, test_x.pcs_m.values, title=title, reference='camp-dist-1-sa', display=False)

In [25]:
# Filter for Scenario 
test_xi = f_combi[(f_combi['situation'] == 2) & (f_combi['distance'] == 1)].copy()
test_xi.rename(columns={'pcs/m²':'pcs_m'}, inplace=True)
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()
test_x = test_x[['fréquentation', 'situation', 'distance', 'substrat', 'pcs_m']]

func = RandomForestRegressor(n_estimators=estimators, criterion="absolute_error", random_state=42)
predictions, feature_importance, mse, r2 = analyze_scenario(test_x, func,  n_iterations=iterations, bin_width=0.2)

# the quantiles for this scenario
caption = 'Urban, <= 500 m du parking'
q_sit_2_d_1 = makeqdf(test_x.pcs_m.values, predictions, caption=caption)
glue('q-ville-dist_1-sa', q_sit_2_d_1, display=False)

# the histogram for this scenario:
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions - Situation Ville, distance < 500 m'
plot_histogram(predictions, test_x.pcs_m.values, title=title, reference='ville-dist-1-sa', display=False)

In [26]:
# Filter for Scenario 
test_xi = f_combi[(f_combi['substrat'] == 1)].copy()
test_xi.rename(columns={'pcs/m²':'pcs_m'}, inplace=True)
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()
test_x = test_x[['fréquentation', 'situation', 'distance', 'substrat', 'pcs_m']]
func = RandomForestRegressor(n_estimators=estimators, criterion="absolute_error", random_state=42)
predictions, feature_importance, mse, r2 = analyze_scenario(test_x, func,  n_iterations=iterations, bin_width=0.2)

# the quantiles for this scenario
caption = 'Sables'
q_sub_1 = makeqdf(test_x.pcs_m.values, predictions, caption=caption)
glue('q_subs_1-sa', q_sub_1, display=False)

# the histogram for this scenario:
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions - Sables'
plot_histogram(predictions, test_x.pcs_m.values, title=title, reference='subs_1_hist-sa', display=False)

In [27]:
# Filter for Scenario 
test_xi = f_combi[(f_combi['substrat'] == 2)].copy()
test_xi.rename(columns={'pcs/m²':'pcs_m'}, inplace=True)
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()
test_x = test_x[['fréquentation', 'situation', 'distance', 'substrat', 'pcs_m']]
func = RandomForestRegressor(n_estimators=estimators, criterion="absolute_error", random_state=42)
predictions, feature_importance, mse, r2 = analyze_scenario(test_x, func,  n_iterations=iterations, bin_width=0.2)

# the quantiles for this scenario
caption='Graviers'
q_sub_2 = makeqdf(test_x.pcs_m.values, predictions, caption=caption)
glue('q_subs_2-sa', q_sub_2, display=False)

# the histogram for this scenario:
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions - Graviers'
plot_histogram(predictions, test_x.pcs_m.values, title=title, reference='subs_2_hist-sa', display=False)

In [28]:
# Filter for Scenario 
# This is all the values => no filter
# just aggregating to the sample_id 
test_xi = f_combi.copy()
test_xi.rename(columns={'pcs/m²':'pcs_m'}, inplace=True)
test_x = test_xi.groupby(cols, as_index=False).pcs_m.sum()

func = RandomForestRegressor(n_estimators=estimators, criterion="absolute_error", random_state=42)
predictions, feature_importance, mse, r2 = analyze_scenario(test_x, func,  n_iterations=iterations, bin_width=0.2)

# the quantiles for this scenario
caption = 'Toutes les conditions'
q_tous = makeqdf(test_x.pcs_m.values, predictions, caption=caption)
glue('q-tous-sa', q_tous, display=False)

# the histogram for this scenario:
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions'
plot_histogram(predictions, test_x.pcs_m.values, title=title, reference='tous-sa', display=False)

(random_forest_sa)=
### Random Forest 

Source : [scikit-learn random forest](https://scikit-learn.org/0.16/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

criterion : `absolute error`

La régression avec forêt aléatoire est une technique d'apprentissage automatique (machine learning) utilisée pour prédire des résultats continus (par opposition aux catégories dans la classification). C'est une méthode d'apprentissage ensembliste, ce qui signifie qu'elle combine les prédictions de plusieurs algorithmes d'apprentissage automatique pour produire des prédictions plus précises.

::::{tab-set}

:::{tab-item} Toutes les conditions
{glue}`tous-sa`
:::

:::{tab-item} Graviers
{glue}`subs_2_hist-sa`
:::

:::{tab-item} Sables
{glue}`subs_1_hist-sa`
:::

:::{tab-item} Ville et haute Fréquentation
{glue}`ville-hf-sa`
:::

:::{tab-item} Campagne et haute fréquentation
{glue}`camp-hf-sa`
:::

:::{tab-item} Campagne et parking <= 500 m
{glue}`camp-dist-1-sa`

:::

:::{tab-item} Ville et parking <= 500 m
{glue}`ville-dist-1-sa`

:::



:::{tab-item} Résultats
:selected:

````{grid} 1 2 2 2

```{grid-item}
{glue}`q-tous-sa`
```

```{grid-item}

Les modèles ont fait l'objet d'un bootstrap, 100 itérations pour chaque scénario. Les résultats estimés sont la collection de toutes les prédictions de chaque itération.

Par exemple, le tableau intitulé "Gravier" présente les résultats observés et prévus pour les plages ayant un substrat de 3 ou 4.

```

```{grid-item}
{glue}`q_subs_2-sa`
```

```{grid-item}
{glue}`q_subs_1-sa`
```

```{grid-item}
{glue}`q-hf-ville-sa`
```

```{grid-item}
{glue}`q-hf-camp-sa`
```

```{grid-item}
{glue}`q-camp-dist_1-sa`
```

```{grid-item}
{glue}`q-ville-dist_1-sa`
```

````
:::

::::
    

In [29]:
from typing import Type, Optional, Callable
from typing import List, Dict, Union, Tuple

def sum_a_b(zipped):
    for element in zipped:
        # the new beta distribution would be
        # total success, (total tries - total success)
        new_element_0 = np.array([np.array([x[0], x[1] - x[0]]) for x in element[0]])
        new_element_1 = np.array([x for x in element[1]])
        t3 = new_element_0 + new_element_1
        
        yield t3

# Grid approximation
grid_val_index = np.linspace(0, 5.99, 600)
groupby_columns = ['sample_id', 'location', 'date', 'city', 'orchards', 'vineyards', 'buildings', 'forest',
                   'undefined', 'public_services', 'streets']
def draw_a_beta_value(generator):
    d = next(generator)
    # drawing a random number from the beta distribution
    # this is the the chance p, that a binomial distribution will
    # result in True.
    my_beta = [beta(x[0], x[1]).rvs(1) for x in d]
    yield my_beta

def binomial_probability_of_failure(generator):
    # in this case failure means exceeding the value
    # for trash a success is never exceeding the value
    d = next(generator)
    di = [x[0] for x in d]
    yield di

def bin_land_use_values(*, data: pd.DataFrame, column: str, num_bins: int = 4) -> pd.DataFrame:
    """
    Bins the specified column's values into a given number of bins and adds a new column to the DataFrame with these bin labels.

    Args:
        data (pd.DataFrame): The DataFrame to modify.
        column (str): The name of the column to bin.
        num_bins (int, optional): The number of bins to use. Defaults to 20.

    Returns:
        pd.DataFrame: The modified DataFrame with an additional column for binned values.
    """
    data[f'{column}_bin'] = pd.cut(data[column], bins=num_bins, labels=[1, 2, 3, 4 ], include_lowest=True)
    return data


def calculate_likelihood(*, aggregated_data: pd.DataFrame, bin_density_column: str, pcs_column: str = 'pcs/m',
                         grid_range: np.ndarray = None, bins: list = None) -> pd.DataFrame:
    """
    Calculates the likelihood of observing the aggregated pcs/m data for each grid point and bin density value.

    Args:
        aggregated_data (pd.DataFrame): The aggregated data to be used for likelihood calculation.
        bin_density_column (str): The column representing bin density numbers.
        pcs_column (str, optional): The pcs/m column to use for calculation. Defaults to 'pcs/m'.
        grid_range (np.ndarray, optional): The range of grid values. Defaults to np.linspace(0, 9.99, 1000).

    Returns:
        pd.DataFrame: A DataFrame with likelihood values for each grid value and bin density number.
    """
    likelihood_df = pd.DataFrame(index=grid_range)
    
    for bin_value in bins:
        bin_data = aggregated_data[aggregated_data[bin_density_column] == bin_value]
        if bin_data.empty:
            likelihoods = [np.array([1, 1]) for grid_point in grid_range]
        else:
            likelihoods = [np.array([(bin_data[pcs_column] > grid_point).sum(), len(bin_data)]) for grid_point in
                           grid_range]
        likelihood_df[f'Likelihood_{bin_value}'] = likelihoods
    return likelihood_df

def calculate_beta_prior(*, grid_range: np.ndarray = grid_val_index, bin_density_numbers: List[int] = list(range(1,
                                                                                                    21))) -> pd.DataFrame:
    """
    Calculates a Beta(1, 1) prior for each value in the specified grid range for each bin density number.

    Args:
        grid_range (np.ndarray, optional): The range of grid values. Defaults to np.linspace(0, 9.99, 1000).
        bin_density_numbers (List[int], optional): List of bin density numbers. Defaults to range(1, 21).

    Returns:
        pd.DataFrame: A DataFrame with Beta(1, 1) prior values for each grid value and bin density number.
    """
    prior_df = pd.DataFrame(index=grid_range)
    prior_values = np.array([1, 1])  # Constant value since Beta(1, 1) is uniform
    
    for bin_number in bin_density_numbers:
        prior_df[f'Bin_{bin_number}'] = [prior_values for grid_point in grid_range]
    return prior_df

In [30]:
from scipy.stats import beta
from scipy.stats import multinomial

def define_posterior(likelihood, prior, grid_val_index: np.array = None):
    
    # the alpha, beta parameters of the likelihood and prior are assembled
    alpha_beta = list(zip(likelihood.values, prior.values))
    a_b_sum = sum_a_b(alpha_beta)
    
    posteriors = []
    for i in grid_val_index:
        # the sum of successes and failures for the scenario at the given
        # grid value are used as the alpha, beta parameters of the beta distribtion
        # for the binomial/bernouli probability that a sample will exceed the grid
        # value i.
        st = binomial_probability_of_failure(draw_a_beta_value(a_b_sum))
        val = next(st)
        posteriors.append(val)
    
    # return posterior probabilities with gird index and column labels
    post_grid_pstock = pd.DataFrame(posteriors, index=grid_val_index, columns=prior.columns)
    
    # identify the x scale of the grid
    post_grid_pstock['X'] = post_grid_pstock.index
    
    # this column is the normalized probabilities that a sample
    # will exceed a value on the grid.
    post_grid_pstock['norm'] = post_grid_pstock['Bin_1']/post_grid_pstock['Bin_1'].sum()
    
    return post_grid_pstock

def non_zero(alist):
    # find the first non-zero object in an array
    # return the index number and the value.
    for i, anum in enumerate(alist):
        if anum != 0:
            return i, anum
    return None

def draw_sample_from_multinomial(normed, n=100):
    # the norm column from the posterior data frame is
    # used as the probabilities of a multinomial distribution
    rv = multinomial(1, normed.values)
    y = rv.rvs(n)   

    indexes = []
    for i in range(0, len(y)):
        indexes.append(non_zero(y[i])[0])
    return indexes


def posterior_predictions(p_g_p):
    
    p_norm = p_g_p['norm']
    
    indexes = draw_sample_from_multinomial(p_norm)
    results_scale = p_g_p.reset_index(drop=True)
    sample_totals = results_scale.loc[indexes, "X"]
    
    return sample_totals

# the prior data from surveys
# iqaasl_prior = report_iq_pk.w_df[report_iq_pk.w_df.project == "IQAASL"].copy()
# iq_p = iqaasl_prior.groupby(['loc_date', 'project'], as_index=False).pcs_m.sum()
# iq_p['top'] = 1

# iq_prior = calculate_likelihood(aggregated_data=iq_p, bin_density_column='top', pcs_column='pcs_m', grid_range=grid_val_index, bins=[1])
# iq_prior.rename(columns={'Likelihood_1':'Bin_1'}, inplace=True)

# assuming no prior knowledge
# that is we assume there is an equal chance (50%) of finding any
# number on the grid 0.00 - 5.99
beta_prior = calculate_beta_prior(bin_density_numbers=[1])
    
col = 'top'
pcs_col = 'pcs/m²'
grid_range = grid_val_index
bins = [1]

test_x = f_combi.copy().groupby(cols, as_index=False)[pcs_col].sum()
test_x['top'] = 1

grid_pstock = calculate_likelihood(aggregated_data=test_x, bin_density_column=col, pcs_column=pcs_col, grid_range=grid_range, bins=bins)

# posterior uninformed
post_grid_pstock = define_posterior(grid_pstock, beta_prior, grid_val_index=grid_range)

# # posterior informed
# post_grid_iqp =define_posterior(grid_pstock, iq_prior, grid_val_index=grid_range)

# samples
sample_totals = posterior_predictions(post_grid_pstock.copy())
# s_iqp = posterior_predictions(post_grid_iqp.copy())

caption = 'Toutes les conditions'

test_grid_quants = makeqdf(test_x[pcs_col].values, sample_totals, caption=caption)
glue('q-tous-b-sa', test_grid_quants, display=False)

title = 'Plastock 2022, Le Léman\nDistribution des Prédictions: toutes les conditions,  grid approximation, prior = IQAASL'

plot_histogram(sample_totals, test_x[pcs_col].values, title=title, reference='toutes-gapprox-sa', display=False)

In [31]:
col = 'substrat'
pcs_col = 'pcs/m²'
grid_range = grid_val_index
bins = [1]

test_xi = f_combi[(f_combi['substrat'] == 1)].copy()
test_x = test_xi.groupby(cols, as_index=False)[pcs_col].sum()


grid_pstock = calculate_likelihood(aggregated_data=test_x, bin_density_column=col, pcs_column=pcs_col, grid_range=grid_range, bins=bins)

# posterior uninformed
post_grid_pstock = define_posterior(grid_pstock, beta_prior, grid_val_index=grid_range)

# samples
sample_totals = posterior_predictions(post_grid_pstock.copy())

caption = 'Sable'

test_grid_quants = makeqdf(test_x[pcs_col].values, sample_totals, caption=caption)
glue('q-sable-b-sa', test_grid_quants, display=False)
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions: Sable, grid approximation'

plot_histogram(sample_totals, test_x[pcs_col].values, title=title, reference='sables-gapprox-sa', display=False)

In [32]:
col = 'substrat'
pcs_col = 'pcs/m²'
grid_range = grid_val_index
bins = [2]

test_xi = f_combi[(f_combi['substrat'] == 2)].copy()
test_x = test_xi.groupby(cols, as_index=False)[pcs_col].sum()


grid_pstock = calculate_likelihood(aggregated_data=test_x, bin_density_column=col, pcs_column=pcs_col, grid_range=grid_range, bins=bins)

# posterior uninformed
post_grid_pstock = define_posterior(grid_pstock, beta_prior, grid_val_index=grid_range)

# samples
sample_totals = posterior_predictions(post_grid_pstock.copy())

caption = 'Graviers'

test_grid_quants = makeqdf(test_x[pcs_col].values, sample_totals, caption=caption)
glue('q-gravier-b-sa', test_grid_quants, display=False)
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions: Graviers, grid approximation'

plot_histogram(sample_totals, test_x[pcs_col].values, title=title, reference='graviers-gapprox-sa', display=False)

In [33]:
col = 'fréquentation'
pcs_col = 'pcs/m²'
grid_range = grid_val_index
bins = [3]

test_xi = f_combi[(f_combi['situation'] == 2) & (f_combi['fréquentation'] == 3)].copy()
test_x = test_xi.groupby(cols, as_index=False)[pcs_col].sum()


grid_pstock = calculate_likelihood(aggregated_data=test_x, bin_density_column=col, pcs_column=pcs_col, grid_range=grid_range, bins=bins)

# posterior uninformed
post_grid_pstock = define_posterior(grid_pstock, beta_prior, grid_val_index=grid_range)

# samples
sample_totals = posterior_predictions(post_grid_pstock.copy())

caption = 'Ville et haut fréquentation'

test_grid_quants = makeqdf(test_x[pcs_col].values, sample_totals, caption=caption)
glue('q-v-hf-b-sa', test_grid_quants, display=False)
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions: Ville et haut fréquentation, grid approximation'

plot_histogram(sample_totals, test_x[pcs_col].values, title=title, reference='v-hf-gapprox-sa', display=False)

In [34]:
col = 'fréquentation'
pcs_col = 'pcs/m²'
grid_range = grid_val_index
bins = [3]

test_xi = f_combi[(f_combi['situation'] == 1) & (f_combi['fréquentation'] == 3)].copy()
test_x = test_xi.groupby(cols, as_index=False)[pcs_col].sum()


grid_pstock = calculate_likelihood(aggregated_data=test_x, bin_density_column=col, pcs_column=pcs_col, grid_range=grid_range, bins=bins)

# posterior uninformed
post_grid_pstock = define_posterior(grid_pstock, beta_prior, grid_val_index=grid_range)

# samples
sample_totals = posterior_predictions(post_grid_pstock.copy())

caption = 'Campagne et haut fréquentation'

test_grid_quants = makeqdf(test_x[pcs_col].values, sample_totals, caption=caption)
glue('q-cam-hf-b-sa', test_grid_quants, display=False)
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions: Campagne et haut fréquentation, grid approximation'

plot_histogram(sample_totals, test_x[pcs_col].values, title=title, reference='cam-hf-gapprox-sa', display=False)

In [35]:
col = 'situation'
pcs_col = 'pcs/m²'
grid_range = grid_val_index
bins = [2]

test_xi = f_combi[(f_combi['situation'] == 2) & (f_combi['distance'] == 1)].copy()
test_x = test_xi.groupby(cols, as_index=False)[pcs_col].sum()


grid_pstock = calculate_likelihood(aggregated_data=test_x, bin_density_column=col, pcs_column=pcs_col, grid_range=grid_range, bins=bins)

# posterior uninformed
post_grid_pstock = define_posterior(grid_pstock, beta_prior, grid_val_index=grid_range)

# samples
sample_totals = posterior_predictions(post_grid_pstock.copy())

caption = 'Ville et distance <= 500 m'

test_grid_quants = makeqdf(test_x[pcs_col].values, sample_totals, caption=caption)
glue('q-ville-d1-sa', test_grid_quants, display=False)
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions: Ville et distance <= 500 m, grid approximation, '

plot_histogram(sample_totals, test_x[pcs_col].values, title=title, reference='v-d1-sa', display=False)

In [36]:
col = 'situation'
pcs_col = 'pcs/m²'
grid_range = grid_val_index
bins = [1]

test_xi =  f_combi[(f_combi['situation'] == 1) & (f_combi['distance'] == 1)].copy()
test_x = test_xi.groupby(cols, as_index=False)[pcs_col].sum()


grid_pstock = calculate_likelihood(aggregated_data=test_x, bin_density_column=col, pcs_column=pcs_col, grid_range=grid_range, bins=bins)

# posterior uninformed
post_grid_pstock = define_posterior(grid_pstock, beta_prior, grid_val_index=grid_range)

# samples
sample_totals = posterior_predictions(post_grid_pstock.copy())

caption = 'Campagne et distance <= 500 m'

test_grid_quants = makeqdf(test_x[pcs_col].values, sample_totals, caption=caption)
glue('q-cam-d1-sa', test_grid_quants, display=False)
title = 'Plastock 2022, Le Léman\nDistribution des Prédictions: Campagne et haut fréquentation, grid approximation,'

plot_histogram(sample_totals, test_x[pcs_col].values, title=title, reference='cam-d1-sa', display=False)

(grid_approx_p)=
### Approximation Bayésienne par Grille

Source : [hammerdirt](https://hammerdirt-analyst.github.io/feb_2024/titlepage.html)

application : [solid-waste-team](https://hammerdirt-analyst.github.io/solid-waste-team/grid_approximation.html)

prior : beta(1,1)

Cas d'utilisation : Cette méthode est une approche manuelle de l'inférence Bayésienne. Elle est particulièrement utile lorsque vous souhaitez incorporer des croyances antérieures et mettre à jour ces croyances avec des données observées.

Mise en œuvre : Implique la définition d'une grille de valeurs de paramètres et le calcul de la vraisemblance des données observées à chaque point de cette grille. En multipliant par la probabilité a priori et en normalisant, on obtient la distribution a posteriori. Cela peut être fait pour chaque condition séparément ou pour toutes les conditions ensemble, bien que cela soit plus intensif en termes de calcul.



::::{tab-set}

:::{tab-item} Toutes les conditions
{glue}`toutes-gapprox-sa`
:::

:::{tab-item} Graviers
{glue}`graviers-gapprox-sa`
:::

:::{tab-item} Sables
{glue}`sables-gapprox-sa`

:::

:::{tab-item} Ville et haute Fréquentation
{glue}`v-hf-gapprox-sa`

:::

:::{tab-item} Campagne et haute fréquentation
{glue}`cam-hf-gapprox-sa`

:::

:::{tab-item} Campagne et parking <= 500 m
{glue}`cam-d1-sa`

:::

:::{tab-item} Ville et parking <= 500 m
{glue}`v-d1-sa`

:::



:::{tab-item} Résultats
:selected:

````{grid} 1 2 2 2

```{grid-item}
{glue}`q-tous-b-sa`
```

```{grid-item}

Prédictions : Fournit une distribution de valeurs possibles de pcs/m, offrant une idée de la fourchette et de l'incertitude des prédictions. Particulièrement utile lorsque la prise de décision nécessite de comprendre l'incertitude ou la variabilité des prédictions.

```

```{grid-item}
{glue}`q-gravier-b-sa`
```

```{grid-item}
{glue}`q-sable-b-sa`
```

```{grid-item}
{glue}`q-v-hf-b-sa`
```

```{grid-item}
{glue}`q-cam-hf-b-sa`
```

```{grid-item}
{glue}`q-cam-d1-sa`
```

```{grid-item}
{glue}`q-ville-d1-sa`
```

````
:::

::::
    

## Citations

\* Proposed citation: Vlachogianni, Th., Anastasopoulou, A., Fortibuoni, T., Ronchi, F.,
Zeri, Ch., 2017. Marine Litter Assessment in the Adriatic and Ionian Seas. IPA-Adriatic
DeFishGear Project, MIO-ECSDE, HCMR and ISPRA. pp. 168 (ISBN: 978-960-6793-25-7)

The DeFishGear marine litter assessment report presents the results of
the one-year long marine litter surveys aiming to assess the amounts,
sources and impacts of marine macro-litter in the Adriatic and Ionian Seas.

\*\* Citation: Yona D, Nooraini P, Putri SEN, Sari SHJ, Lestariadi RA and Amirudin A (2023) Spatial distribution and composition of marine litter on sandy beaches along the Indian Ocean coastline in the south Java region, Indonesia. Front. Mar. Sci. 10:1220650. doi: 10.3389/fmars.2023.1220650

This study analyzed beach litter composition on five beaches (Kondangmerak, Balekambang, Ungapan, Ngudel, and Goa Cina) along the Indian Ocean coastline in the eastern part of the south Java region, Indonesia.

\*\*\* Vlachogianni, Th., 2019. Assessing marine litter on Mediterranean beaches. Filling in the knowledge
gaps via a participatory-science initiative. MIO-ECSDE.

The beach litter surveys were carried out on beaches located in five Mediterranean countries,
namely in Croatia, Cyprus, France, Greece and Italy. A total of 23 sites were surveyed and two sets of
surveys were performed; from mid-September to mid-October 2018

In [37]:
report_data

,échantillon,Plage,substrat,date,area,slug,code,quantité,pcs/m²,region,city,groupname
1531,"('cully', '04.05.2022')",Cully,3,2022-05-04,67,cully-p,G113,0,0.000000,Haut lac,Cully,micro plastics (< 5mm)
1532,"('cully', '04.05.2022')",Cully,3,2022-05-04,67,cully-p,G114,0,0.000000,Haut lac,Cully,micro plastics (< 5mm)
1533,"('cully', '04.05.2022')",Cully,3,2022-05-04,67,cully-p,G148,0,0.000000,Haut lac,Cully,packaging non food
1534,"('cully', '04.05.2022')",Cully,3,2022-05-04,67,cully-p,G27,0,0.000000,Haut lac,Cully,tobacco
1535,"('cully', '04.05.2022')",Cully,3,2022-05-04,67,cully-p,G30,0,0.000000,Haut lac,Cully,food and drink
...,...,...,...,...,...,...,...,...,...,...,...,...
1207,"('tougues', '29.01.2022')",Tougues,2,2022-01-29,544,tougues,G27,36,0.066176,Petit lac,Tougues,tobacco
1208,"('tougues', '29.01.2022')",Tougues,2,2022-01-29,544,tougues,G30,8,0.014706,Petit lac,Tougues,food and drink
1209,"('tougues', '29.01.2022')",Tougues,2,2022-01-29,544,tougues,G31,3,0.005515,Petit lac,Tougues,food and drink
1210,"('tougues', '29.01.2022')",Tougues,2,2022-01-29,544,tougues,Gfoams,3,0.005515,Petit lac,Tougues,infrastructure


## Inventaire

In [38]:
inventory['index'] = inventory.index.map(lambda x: codes.fr.loc[x])
inventory.reset_index(inplace=True)
inventory.set_index(['code','index'], drop=True, inplace=True)
inventory.index.name = None
inventory.sort_values(by='Quantité', inplace=True, ascending=False)
inventory.style.set_table_styles(table_css_styles).format(**format_kwargs)

,,Quantité,% du total,pcs/m²,Taux d'échec
code,index,,,,
Gfrags,"Fragments de plastique: G80, G79, G78, G75",11'221,"0,41","0,26","0,97"
G27,Mégots et filtres à cigarettes,3'089,"0,11","0,07","0,79"
G30,"Emballages de bonbons, de snacks",2'080,"0,08","0,05","0,74"
G106,Fragments de plastique angulaires <5mm,1'926,"0,07","0,03","0,41"
G112,Pellets industriels (GPI),1'526,"0,06","0,02","0,36"
Gfoams,"Fragments de polystyrène expansé: G76, G81, G82, G83",1'399,"0,05","0,05","0,72"
Gcaps,"Couvercles en plastique bouteille: G21, G22, G23, G24",1'070,"0,04","0,03","0,65"
G95,Coton-tige,1'040,"0,04","0,03","0,54"
G74,Mousse de plastique pour l'isolation thermique,406,"0,01","0,01","0,38"


In [39]:
%watermark --iversions -b -r

Git repo: https://github.com/hammerdirt-analyst/plastock.git

Git branch: dec20

seaborn   : 0.12.2
numpy     : 1.24.2
pandas    : 2.0.0
matplotlib: 3.7.1

